In [1]:
#step 1
import json

import pymongo
from pymongo import MongoClient

print ("pymongo version:", pymongo.version)

pymongo version: 3.11.0


In [2]:
# files locations
data_folder = r"../../Dataset/yelp"
fixed_business_json_file = data_folder + "/json/fixed_yelp_academic_dataset_business.json"
fixed_review_json_file = data_folder + "/json/fixed_yelp_academic_dataset_review.json"
fixed_user_json_file = data_folder + "/json/fixed_yelp_academic_dataset_user.json"
fixed_checkin_json_file = data_folder + "/json/fixed_yelp_academic_dataset_checkin.json"
fixed_photos_json_file = data_folder + "/json/fixed_photos.json"

In [2]:
#step 2
# establing connection
try:
    mongo_client = MongoClient("localhost", 27017)
    # mongo_client = MongoClient('mongodb://localhost:27017')
    print("Connected successfully!!!")
    
    host_info = mongo_client['HOST']
    print ("\nhost:", host_info)
except:
    print("Could not connect to MongoDB")

Connected successfully!!!

host: Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'HOST')


In [3]:
#step 3
db = mongo_client.yelpDB

In [5]:
def import_data_into_collection(raw_path: str, collection) -> None:
    with open(raw_path, "r", encoding="utf-8") as rf:
        for line in rf:
            if len(line.strip()) > 0:
                json_node = json.loads(line)
                collection.insert_one(json_node)

In [4]:
#step 4
# Create collections
user_collection = db.user
business_collection = db.business
review_collection = db.review
checkin_collection = db.checkin
photos_collection = db.photos

In [16]:
# import json data in collections (execute line one by one and only once)

# import_data_into_collection(fixed_user_json_file, user_collection)
# import_data_into_collection(fixed_business_json_file, business_collection)
# import_data_into_collection(fixed_review_json_file, review_collection)
# import_data_into_collection(fixed_checkin_json_file, checkin_collection)
# import_data_into_collection(fixed_photos_json_file, photos_collection)

In [5]:
#step 5
print("Data in user collection: ", user_collection.count_documents({}))
print("Data in business collection: ", business_collection.count_documents({}))
print("Data in review collection: ", review_collection.count_documents({}))
print("Data in checkin collection: ", checkin_collection.count_documents({}))
print("Data in photos collection: ", photos_collection.count_documents({}))

Data in user collection:  1968703
Data in business collection:  80455
Data in review collection:  5508465
Data in checkin collection:  77684
Data in photos collection:  194119


In [18]:
# user_collection.remove()
# business_collection.remove()
# review_collection.remove()
# checkin_collection.remove()
# photos_collection.remove()

In [ ]:
# Printing the data inserted 
cursor = user_collection.find()

for record in cursor: 
    print(record) 

In [ ]:
cursor = user_collection.find().limit(1)
for record in cursor: 
    print(record) 

In [9]:
cursor = business_collection.find().limit(1)
for record in cursor: 
    print(record) 

{'_id': ObjectId('5f49385629d02dcddf10198d'), 'business_id': 'b-pQeaRpvuhoEqudo3uymHIQ', 'name': 'The Empanadas House', 'address': '404 E Green St', 'city': 'Champaign', 'state': 'IL', 'postal_code': '61820', 'latitude': 40.1104457, 'longitude': -88.2330726, 'stars': 4.5, 'review_count': 5, 'is_open': 1, 'attributes': {'RestaurantsAttire': "u'casual'", 'RestaurantsTakeOut': 'True', 'BusinessAcceptsCreditCards': 'True', 'NoiseLevel': "u'quiet'", 'GoodForKids': 'False', 'RestaurantsReservations': 'False', 'RestaurantsGoodForGroups': 'False', 'BusinessParking': 'None', 'RestaurantsPriceRange2': '1', 'HasTV': 'True', 'Alcohol': "'none'", 'BikeParking': 'False', 'RestaurantsDelivery': 'True', 'ByAppointmentOnly': 'False', 'OutdoorSeating': 'False'}, 'categories': 'Restaurants, Ethnic Food, Food Trucks, Food, Argentine, Specialty Food, Imported Food, Empanadas', 'hours': {'Monday': '11:30-14:30', 'Tuesday': '11:30-14:30', 'Wednesday': '11:30-14:30', 'Thursday': '11:30-14:30', 'Friday': '11:3

In [ ]:
cursor = review_collection.find().limit(1)
for record in cursor: 
    print(record) 

In [50]:
def extract_unique_attributes(result):
    unique_attrs: Set[str] = set()
    for record in result:
        if record['attributes'] is not None and len(record['attributes']) > 0:
            cur_attrs = []
            for k, v in record['attributes'].items():
                cur_attrs.append(k)
#             print(cur_attrs) 
        unique_attrs.update(cur_attrs)
    return unique_attrs

In [25]:
# Filtered restaurant (Indian, Pakista)
regex_query = { "$and":[
    {"categories" : {"$regex" : "Restaurant"} },
    {"$or": [
        {"categories" : {"$regex" : "Indian"} },
        {"categories" : {"$regex" : "Pakistani"} }#,
        #{"categories" : {"$regex" : "Korean"} }
    ]}
]}

In [ ]:
filter_rest = business_collection.find(regex_query)
print("Total Filter businesses: ", filter_rest.count())

In [ ]:
for record in filter_rest: 
    print(record) 

In [6]:
#step 6
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")

In [7]:
#step 7
# Getting list of all 'Silver Spoon' restaurants
# Filtered restaurant (Indian, Pakista)
regex_query = { "$and":[
    {"categories" : {"$regex" : "Restaurant"} },
    {"$or": [
        {"categories" : {"$regex" : "Indian"} },
        {"categories" : {"$regex" : "Pakistani"} }#,
        #{"categories" : {"$regex" : "Korean"} }
    ]},
    {"review_count": { "$gt": 10 } }
]}

regex_query_silver = { "$and":[
    {"categories" : {"$regex" : "Restaurant"} },
    {"$or": [
        {"categories" : {"$regex" : "Indian"} },
        {"categories" : {"$regex" : "Pakistani"} }#,
        #{"categories" : {"$regex" : "Korean"} }
    ]},
    {"name" : {"$regex" : "Silver Spoon"} },
]}

# get filterd restaurants
filter_rest = business_collection.find(regex_query).limit(46)
print("Total Filter businesses: ", filter_rest.count())

# get filterd Silver Spoon restaurants
filter_rest_silver = business_collection.find(regex_query_silver)
print("Total Filter Silver spoon businesses: ", filter_rest_silver.count())

# Concate both filter results
restaurant_list = list(filter_rest)
restaurant_list.extend(list(filter_rest_silver))
print("Total Filter businesses: ", len(restaurant_list))

dataset = {}
restaurants = []
all_users = []
all_attrs = set()

# process filtered restaurants to get dataset
for record in restaurant_list: 
    # making restaurant object
    restaurant = {}
    restaurant["rest_id"] = record["business_id"]
    restaurant["name"] = record["name"]
    restaurant["address"] = record["address"]
    restaurant["postal_code"] = record["postal_code"]
    restaurant["city"] = record["city"]
    restaurant["state"] = record["state"]
    ### getting country by lat lng ###
    Latitude = record["latitude"]
    Longitude = record["longitude"]
    location = geolocator.reverse(str(Latitude)+","+str(Longitude))
    address = location.raw['address']
    country = address.get('country', '')
    restaurant["country"] = country
    ######
    restaurant["rating"] = record["stars"]
    
    # getting all attribute list of the restaurant to make Aspects
    attrs = []
    if record["attributes"]:
        for key, value in record["attributes"].items():
            attrs.append(key)

        # update all_attrs set() for unique attribute objects
        all_attrs.update(attrs)

    # fetch all reviews of the restaurant to add in review list
    restaurant_reviews = review_collection.find({'business_id': restaurant["rest_id"]})
    
    # process review object
    reviews = []
    review_limit = 50
    review_count = 0
    for restaurant_review in restaurant_reviews:
        
        # maximum 50 reviews to be select
        review_count += 1
        if review_count > 50:
            break
        
        review = {}
        review["review_id"] = restaurant_review['review_id']
        review["user_id"] = restaurant_review['user_id']
        review["date"] = restaurant_review['date']
        review["rating"] = restaurant_review['stars']
        review["text"] = restaurant_review['text']
        
        # get reviewer object from user table to get the name of reviewer and make user list
        user = {}
        user_obj = user_collection.find({'user_id': review["user_id"]})
        
        # set reviewer name in review object
        review["name"] = user_obj[0]['name']
        # add review in review list
        reviews.append(review)
        
        # make user object
        user['user_id'] = user_obj[0]['user_id']
        user['name'] = user_obj[0]['name']
        user['review_count'] = user_obj[0]['review_count']
        
        # make list of user object and add in user list
        user_friends = user_obj[0]['friends'].split(',')
        user['friends'] = user_friends
        all_users.append(user) 
    
    # add reviews in restaurant object
    restaurant["reviews"] = reviews
    
    # add restaurant in restaurant list
    restaurants.append(restaurant)
    
# make unique user list
users = list({user['user_id']:user for user in all_users}.values())

# add restaurant, users and aspect list in dataset
dataset["restaurants"] = restaurants
dataset["users"] = users
dataset["aspects"] = list(all_attrs)

# remove user friends that are not in users list
user_ids = set()
for user in dataset["users"]:
    user_ids.add(user["user_id"])

for user in dataset["users"]:
    friends_exist_arr = list(filter(lambda x: x in user_ids, user["friends"]))
    user["friends"] = friends_exist_arr

# Serializing json and write to a file
# json_dataset = json.dumps(dataset, indent = 4)
# print(json_dataset)
with open('dataset.json', 'w') as json_file:
    json.dump(dataset, json_file)

<ipython-input-7-122a487e7632>:25: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print("Total Filter businesses: ", filter_rest.count())
<ipython-input-7-122a487e7632>:29: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print("Total Filter Silver spoon businesses: ", filter_rest_silver.count())


Total Filter businesses:  1107
Total Filter Silver spoon businesses:  4
Total Filter businesses:  50


In [17]:
import re
from tqdm import tqdm

regex = re.compile(r'[\n\r\t]')
all_review_text = review_collection.find({}, {"text":1})
total_reviews = all_review_text.count()
print("Total Filter Reviews: ", total_reviews)

i = 0
for line in tqdm(all_review_text, total=total_reviews):
    trimmed_line = regex.sub('', line["text"].strip())
    i = i+1
    if trimmed_line:
        with open("all_reviews.txt", 'a') as f_write:
            f_write.write(trimmed_line)
            f_write.write("\n")
            f_write.close()
    
#     if i == 3:
#         break

<ipython-input-17-25f43805dcc6>:6: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  total_reviews = all_review_text.count()
  0%|          | 102/5508465 [00:00<1:49:15, 840.23it/s]

Total Filter Reviews:  5508465


100%|██████████| 5508465/5508465 [02:45<00:00, 33335.60it/s]


In [11]:
rev_count = 0
for f_rest in filter_rest:
    rev_count = rev_count + f_rest["review_count"]
print(rev_count)

92252


In [16]:
i = 0
print(filter_rest.count())
for f_rest in filter_rest:
    print("["+str(i)+"] "+f_rest["business_id"])
    filter_rev = review_collection.find({"business_id":f_rest["business_id"]})
    print(filter_rev.count())
    i = i+1
    for f_rev in filter_rev:
        f = open("reviews_file.txt", "a")
#         text = f_rev["text"].rstrip('\n')
#         text = text.replace('\t','')
        f.write(f_rev["text"])
        f.close()


<ipython-input-16-bc1b5537cb94>:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print(filter_rest.count())
<ipython-input-16-bc1b5537cb94>:6: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print(filter_rev.count())


1722
[0] b-RrapAhd8ZxCj-iue7fu9FA
3
[1] b-Yy_SB9r7VQTlIVv_MeOmaA
9
[2] b-LZWXP-D4YPlzsFjVx6b9XA
20
[3] b-CvOxwAhmwwy3sQRKYVAjGg
22
[4] b-IjYGSm4_DjP6VULpP27o_g
10
[5] b-ZDx7kt4aOPTlmYTqXDrTGA
15
[6] b-p-J9iJgeHmPGk2jFAM0WnA
23
[7] b-RE4qn28MEiDrM1PbdYVgxA
20
[8] b-9j4nk2WOT6XCDRgX-lGzVA
6
[9] b-_PKXarw3GjlbwbXhjdpUMA
29
[10] b-scM6rLQqoz5dEIgwz_JFsg
144
[11] b-eRZuxjnaMviiVyvhejwTBQ
32
[12] b-yv4Ov5b21VI_-52BNRLolA
3
[13] b-Mdk1C_CBlKjX1sfH-xLIBg
3
[14] b-iDSHlWunyWMS02P8kGyHJQ
7
[15] b-lVaHDON2mxLr5mneUnawKg
9
[16] b-CB99Ifh4OnBTnuNdRjj_fQ
28
[17] b-Eu-M3y8a5bIxIRXlCj9yHA
201
[18] b-0kKtYDKvvcQckR4vRmfh5Q
4
[19] b-Dh2Z3WNu6M71LbMNHEeg3w
16
[20] b-TTCZkfxt0GW8YUzdVe0PrA
74
[21] b-lBwx8insD-RNa-_lOtA3kA
6
[22] b-pbnZqw5tHkZZBG5Ryys0yw
6
[23] b-or1daHtUK9b7sqFRPf62GQ
13
[24] b-vZu3xxlMQSqFngDgdFLfXg
16
[25] b-5eDg_Y2N6aU_k7tIxoFXTg
62
[26] b-x8SLNG0wEaGD3SrrZVMDcg
5
[27] b-3wJ4kJsJhayyfJ30HsS04g
20
[28] b-qhKNZTtTT6GLnPzQny-v9Q
28
[29] b-k9PfKE7KfztoMpOSx25UpA
37
[30] b-3frFidAcPVYAFB2Rx

141
[246] b-k9byY47rtCCXaVPo9GyYPg
4
[247] b-86d1M8MMrb94n5djgKWqEQ
61
[248] b-92g7A0sCbb7VklNyoJphew
7
[249] b-112pGcb0ZB666d_IwFt4jA
73
[250] b-OJfFKXx0AHk5VsL5VPvbdA
27
[251] b-lO-IY5X36bLI8vnQR4O18A
52
[252] b-QOZQeWCVgbs4-HYZjAXpCg
13
[253] b-_-mGxo3NxooLN7DQWlvTYA
3
[254] b-7BIbvCkI7AXQMVC1-DHpCA
19
[255] b-1Z8uTtx73d1JIRTCCcsQeg
10
[256] b-I6DyyWnsULTNrm5X2ABnnQ
4
[257] b-9_K9P144hsMNZp_1rIGzLg
3
[258] b-AdQvlunq29WGZVvQX_PAYQ
3
[259] b-irCBE0vRZ9oQZ26qMR-5HA
25
[260] b-eUpav2QqCcLXZt9hKSBiuw
13
[261] b-AYio05fi_8aEYOo43JEqfg
18
[262] b-d6XiEFppPRpj5AZk_VzWZA
30
[263] b-XZxJ4fG-4a3PP9VP5jvDWw
201
[264] b-wPl5OzBIXBCAfhGnymhjXw
63
[265] b-YUqejVW1bj_gjJbMvOEusQ
58
[266] b-RKQ0qEIEP-VdS8X4YMa2IA
58
[267] b-Yh5j6ESKrXBoIuOtLrkBfw
6
[268] b-2-Y-kNEbrP_m1U9Hc2xH7A
21
[269] b--JDAieYuZocQSVYGqwWKog
23
[270] b-MWffCS7UkTUj9yDMy4jaLA
21
[271] b-8PCFk6ZKDm73Ag42C8XtGw
44
[272] b-7m8MoWuudrxU98fEw7iBIw
17
[273] b-PlJ5Tf9T4QlFQoNZf99aTg
27
[274] b-mxtMK3cHhtvYGcI1nozpWA
6
[275] b-X2oBZQaFy

12
[489] b-OFAiA6En2HD0CqLkFDCNpg
111
[490] b-Ijh1AcbH-F3CyhYT_5qL8Q
79
[491] b-o8k9D05WpwgK-8BV7OH-xw
26
[492] b-KfxPrJ5PzgFRm2XltnbBLQ
256
[493] b-kEIe_lyBKdFB6Hv_wsUYaw
81
[494] b-qiZ7t01x0xqqB0RIvO3yuQ
61
[495] b-v38zwJvJiLv-yC7uEhhcKA
9
[496] b-U8tIgXZ0T--8ZNEVoDap0g
27
[497] b-8e7hNOW63bd5ce_RdMDUNw
9
[498] b-RkOguMKU13hPglELb6t2Vg
215
[499] b-DN_cXMaeAUFFLm6eUOsNWQ
37
[500] b-br3DuzzRsyBX9VcLWtEh_w
15
[501] b-XL33UIY33sl3eyr1hyDxMQ
70
[502] b-5OHW43gqdoWOGWDmUmps6A
26
[503] b-o_sq_rvTBo59hm6TkDi6tg
67
[504] b-VaWzc9Y3WvI6qVh0zOBzjA
3
[505] b-KvRh-exPg_KegOM_HXeskw
133
[506] b--WKflOYB9SBUMz8DgFzj6Q
72
[507] b-uRbLTO_32hBxG7s7LykwlQ
3
[508] b-q80WmfxmifLBdB8_rPYu1w
13
[509] b-4yx_HHr_o4z4KjbJJOtqSw
5
[510] b-nGGHX9HSHHWTyTCaPVxU7g
5
[511] b-kTJEf56gjzBSpi9X14X54Q
23
[512] b-hp1qK1gOvYRdhhBQsE-zLg
14
[513] b-8HK_pj0_EJUykVAD4zHa5w
4
[514] b-HF-fLG3QaNepY151QKDNLA
30
[515] b-XrcB7Of1KfHlkeXYpHLIrw
248
[516] b-S1BC9GXZBOIT3rO-uWifVw
3
[517] b-ECtQU_nN7OX1xnz1gujGcQ
123
[518] b-daI7_

23
[732] b-yi1LdN8dBh3TLN1hTTOrGw
47
[733] b-UuEfwtPMT9W9tgDmI4o7tQ
89
[734] b-rROHsa0BQsKjxNnfYjBF9g
92
[735] b-ad7lp1JL-gC4d4zDXXNgjg
20
[736] b-33yClygMVQZb5y9WgOCHLA
38
[737] b-PGde9ygfV_a5oLQDNVedJg
41
[738] b-oItcB1TAOsbBp3qYJzSHOQ
5
[739] b--XNodvHlq9oDu-kQBwGFkg
11
[740] b-RXyBD1euUHNxhU3_LE7z6Q
11
[741] b-Oi_VMdVbXHuyLi2gxjqd5Q
18
[742] b-694ZSZQfbpycmzyMQm-XGw
30
[743] b-gJm6XxUQ7uHlrh8Y8fRiuw
166
[744] b-0v8icS8wOOgEDiHDCOQkZQ
374
[745] b-Y_G2R89DBoiYusDI8H9Yjw
14
[746] b-biVQDp3K_vNP1eK5wGZOJg
151
[747] b-iE71iwcSljg3xm2GB2Y9aA
615
[748] b-yKKSH29Lym9Ys9li2Lcnqg
17
[749] b-7utCm6_uBRnu0lgLV82LaQ
12
[750] b-BLcvgDX3gGrymH4D6jlWfA
51
[751] b-_Oe8NDjmeYkkHFs7f8CekA
19
[752] b-HpT0pKeu-HeBtjxkrqxv8w
4
[753] b-jte_XM9Zo7TX86jlKwq5gg
112
[754] b-Q8GdWF9qLNf0aOBXaKUEPg
11
[755] b-dNdKrKp7H1xtOxkONkXjkg
18
[756] b-hFay_o8K1YfqFiMFoSPv5Q
69
[757] b-ABgeUaVsMEqkcOI4bWsj2Q
18
[758] b-3Dr8WhMdmxYfGHmvCnXzmw
18
[759] b-WRVPOzgybjAtZJGn_0ucSg
12
[760] b-IxImEiJocfQHMl_xphWYag
5
[761] b-u

102
[975] b-kQIU9E7yHwvFcNRd8taYyA
6
[976] b-hdQK1rfLEjwRlCppLFo10g
59
[977] b-ARp5inQqiKuDt2E9tIrX2Q
9
[978] b-c_ZDTkfDlfzLm0a3osk1ow
24
[979] b-yS16AkIQj5gwSfk2bBaxkQ
8
[980] b-TfabYPasZ1oOMN6JLa-yvw
910
[981] b-ifAv4IoaHVkQyrcE-L51Lw
8
[982] b-MmwKWBBrR0NF05hGBxnWjA
22
[983] b-3HAPJH0Qoi8Ix6M9rUMfjg
49
[984] b-aFMRZBovrBWa8vGzR5TjMw
6
[985] b-0C4b8LsgLuL7_DuPXE729A
11
[986] b-esT5Nnlnr2SnmsH1P-8n-A
36
[987] b-tqJGBUKMNoUNX6FvHOHQGg
40
[988] b-2B4bC4w4PDC8Pj53SfTG2g
101
[989] b-OGSWuGLLW_8TBmQ_wWhdEQ
5
[990] b--HrD-xfoQx9xyYPqNGEdyQ
8
[991] b-OhuICfAfW4fTBn6JmjBC-w
22
[992] b-TiQBE6BdjSg9MF3UCvjXrw
3
[993] b-Mv6ripiINornDXI07VPaLA
52
[994] b-iJfS02AOptwkaWD-vOVRew
29
[995] b-UQMQDq8yuxnyQyBIoKhzoQ
160
[996] b-7NurnM6G9b_2jTEKpIkQpg
66
[997] b-yMqvTSsHkovlPHV8nDn0Ng
3
[998] b-Vu_BoGOSL70odzqCBr_vIg
22
[999] b-FudpYlLuoqeetWX6UCt3PQ
98
[1000] b-U-C7FKpKMdt9VOTXcmS4Hw
86
[1001] b-UadX71pjUtIDHaR61qTx3w
6
[1002] b-cuXCQM-9VwpZlSneEY1b3w
71
[1003] b-3S8nDh3gK2BmG-_HjDjh6w
41
[1004] b-LK3H

[1211] b-Mld11EXvzf9z5msj3ycI0g
7
[1212] b-BiM04hM-1x3OfVJkxoucUQ
42
[1213] b-U7sTaVuXqgMSdCiNb7xdxA
11
[1214] b-ou2dLeaR4soTd_6ZktsPOA
17
[1215] b-v4ZVAIDgE9ZXhMcpx8tctw
14
[1216] b-v3BLVPLRo2Egeak29B9lug
158
[1217] b-lPwUrkMDlaWX_eRhCgxpOA
3
[1218] b-iU7rCNlmoQvp-JZ5dOx96w
94
[1219] b-3r5FidzFQ9rfAoQOtixfxA
22
[1220] b-clq6ziplldA2D19RlEUSxQ
6
[1221] b-4xtU8eji_G0Yepj3D-8I6Q
196
[1222] b-D0WAHGe90EkRKkZwR52cpA
4
[1223] b-Ra6dcmjOIqDdBXremeGEuQ
92
[1224] b-HcqI_ONIaXwzYR7cxFpdUw
4
[1225] b-A171xHIiEuAtyJo7rlRr7w
3
[1226] b-X9sDrd6Uia0uQFFKwwQcJg
14
[1227] b-nKpBO-XDKv_n6n-JR2Z1Jg
11
[1228] b-9bbuOf0Ey8lEVI3gUezviw
13
[1229] b-vpEGCNcfYe_R4blmV2GPCA
39
[1230] b-16-gsPZnaBJgFi9Le-i27w
39
[1231] b-mRTmgaPH6AaceHh7Ci6Mag
6
[1232] b-ScdtR67jO-p_ZIpVbZPRfw
287
[1233] b--YxR1MgbIe3k__YBdbLwXw
142
[1234] b-8ZcZvOcUZ3nT0Q5LvHwQEA
7
[1235] b-2KkTzafNqkEKr5VTjsR3hw
8
[1236] b-xNK43KL8NRnoN9azyyrJ6A
21
[1237] b-pHSHm72bwR09Mt022CVvrw
346
[1238] b-7WM7pwCwsn81yG4FgqFG3w
5
[1239] b-vs5tgU7fn7V87awW

10
[1447] b-1L8QhmM-0dhUHw7sWntVaA
44
[1448] b-7SBtCKKeHbdB6-dIIkcZXw
7
[1449] b-lrv5YU-DUI6DhtW_iX1vyA
5
[1450] b-BmFStvt6hs77XmYR7zmHtQ
18
[1451] b-8td1xV3B0hJc3wYpGmjtRQ
109
[1452] b-94p6Q1TWTqoa39RGDdAHgA
15
[1453] b-cUnIqvSKiZzr1Gw0OATMdA
3
[1454] b-49YU3rvCpen841tnpsEVqQ
4
[1455] b-GYmdm_Vy7sf9tji7WUcohg
50
[1456] b-_LSwdNmiFSzRZQfG4g-qwA
94
[1457] b-Ph29r03lKu-dRRuiWnM7-A
17
[1458] b-3quopZyL8WuqVFfsCi0z2A
95
[1459] b-N-L7ISdndWsIWdK0fn29wg
206
[1460] b-0oOTrzxhO3FxHqmeZuC3mw
39
[1461] b-VWOh8q-OdDR1t3hoPFy71A
9
[1462] b-nyEL20vR0RZ3IUI-Z4m1yg
6
[1463] b-zP-ACMba8Jscn0aAnM4w2w
3
[1464] b-ZqsYnJOa3Bt5ZSLyppDNng
4
[1465] b-9K2FHkajUaTU4P2cfFJmGQ
66
[1466] b-ESuSLeXtEnEf1zs8WiW-Tg
3
[1467] b-IeH7I-29vKD5w9v9pX5KYw
8
[1468] b-9coZa838egYONxSdTe-wUg
31
[1469] b-aDWOXQVSvRPOpqcJ1ja0_Q
79
[1470] b-c65l1HepL-5Zqh_2fJ7ezw
19
[1471] b-Hz6pXloEBWboBaJuEpk0JQ
48
[1472] b-lusyQapYBD-df6VofjRWMQ
36
[1473] b-rqLksDzegxNXMWwZk3eZnQ
165
[1474] b-9wQHmZrkx0MtEN2AlZ0HVw
43
[1475] b-ajHLLUe-A9dTd7W

70
[1683] b-8Zjij16AkcnokBQ2w6yXwg
22
[1684] b-_DiGwfdiH913E4LURwb2qg
193
[1685] b-BjO1LhOE1yH4j3-XfAckAw
4
[1686] b-ocgIMnZ5J-zOhTxBkRIgHg
6
[1687] b-tNPWi75s0o20rNERLg6R_Q
9
[1688] b-FLTgadyh5s8JIBwgR_xxyA
3
[1689] b-xtYiHTmunjfCN2sUaQxBjA
2951
[1690] b-t58AtYOZqw9BlDdZHUrZbA
5
[1691] b-Q7dGYPVFr2HTTdgl5aZDQQ
5
[1692] b-FB-laNy21q5DKmT-cB0G8Q
20
[1693] b-0nJgoF9hAi6f2K7XInr1hQ
28
[1694] b-0anXSUX9KG5sQEzs-Of8Vg
18
[1695] b-7lbkJestGA1bHqgM0znf7w
8
[1696] b-DXsV1EmzKcsnz8bzCHyqnQ
77
[1697] b-3K9u2FpHG9b7tWogqyEyWQ
145
[1698] b-kkXmJbAzH3PlxSVRWcNiyg
25
[1699] b-51yM2HV5lsE8Feuhvs-pGg
36
[1700] b-6IgYs8SiKA0_zx-hF2PVPA
6
[1701] b-rgG2k-8qHXuO4ZKhC1AmIA
73
[1702] b-muInTkijds_uH2uFzNMaNw
3
[1703] b-1FVEkpX23fvZ5gNUs_j6Sg
16
[1704] b-dckVjwfxYLVjDH5161zglA
5
[1705] b-kzV89ZZLFuokHkBF4rvtuA
15
[1706] b-SaAtDmMKN_B5-8d-n5X86w
12
[1707] b-rOs1fCTOfZSS8hXaQplmAw
3
[1708] b-D-eQ32es1ZSTKkE34cf26A
5
[1709] b-hL3LFrwR1UQ-d0CcYF73Sg
8
[1710] b-GtrrBtrfZgvqGF930JwYOg
134
[1711] b-yfHYCkQaupRzWU8F

In [37]:
# exporting attributes from business
business_collection = db.business

cursor = business_collection.find(projection={'business_id':1,'attributes':1})

# print([document['business_id'] for document in cursor])
# for record in cursor: 
#     print(record) 

In [51]:
unique_attributes = extract_unique_attributes(cursor)
print(unique_attributes)

{'DietaryRestrictions', 'RestaurantsTableService', 'RestaurantsAttire', 'HappyHour', 'RestaurantsDelivery', 'Alcohol', 'BusinessAcceptsBitcoin', 'AcceptsInsurance', 'OutdoorSeating', 'GoodForMeal', 'WheelchairAccessible', 'RestaurantsGoodForGroups', 'Music', 'HairSpecializesIn', 'RestaurantsCounterService', 'Ambience', 'Corkage', 'ByAppointmentOnly', 'GoodForDancing', 'Caters', 'BYOB', 'AgesAllowed', 'DriveThru', 'RestaurantsTakeOut', 'BestNights', 'NoiseLevel', 'WiFi', 'DogsAllowed', 'BusinessAcceptsCreditCards', 'BusinessParking', 'RestaurantsReservations', 'BikeParking', 'GoodForKids', 'BYOBCorkage', 'RestaurantsPriceRange2', 'Smoking', 'HasTV', 'Open24Hours', 'CoatCheck'}
